In [1]:

import sys
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
from os.path import isfile,join
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
#import doc2vec_impl
import gensim

import FileProcess as fp



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Total Words in DataSet: 400000
shape of data: (866842, 5)
Started
time-taken: 4.76837158203125e-06


In [2]:
print (fp.window_size)
batch_size=10
vocabulary_size=400000
embedding_size=200
embedding_filename = "glove_6B_200d.txt"
training_epochs=1
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]


2


In [3]:
dirpath = "corpus/fulltext/"
files = [f for f in os.listdir(dirpath) if isfile(join(dirpath, f))]

print(fp.get_sentences(files[0]))

['  1 on 17 april 2008 the application brought by the university of western australia (uwa) against dr bruce gray and sirtex medical ltd (sirtex) was dismissed.', 'uwa was ordered to pay the costs of dr gray and sirtex.', "a cross-claim brought by sirtex against uwa was dismissed and sirtex ordered to pay uwa's costs of that cross-claim.", 'sirtex succeeded in its cross-claim against dr gray.', '2 paragraphs 14 and 15 of the orders made on 17 april 2008 provided:    14.', 'any party in respect of whom an order for costs has been made in the preceding orders is at liberty to file and serve written submissions on or before 8 may 2008 seeking a variation of the costs order.', '15.', 'any party who wishes to respond to a written submission filed pursuant to the preceding order is to do so by filing and serving a written submission by 29 may 2008.', ' 3 on 12 may 2008 sirtex filed a motion seeking the following orders:    1.1 that order 2 of his honour justice french, made on 17 april 2008,

In [4]:
stop_words_list=set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

##########loading dev2vec=============
#doc2vec_impl.main();

d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec_models/doc2vec.model')
#start testing
#printing the vector of document at index 1 in docLabels
docvec = d2v_model.docvecs[files[0]]
print(docvec)


[-1.9957069  -1.7302711  -1.2416555  -1.25041    -0.8760755   1.4730991
  1.2059473   0.38232905 -1.6187527   1.305874    1.1770602   2.308416
  0.02343252 -0.2502587   1.7815354  -1.332049    0.38943136 -2.052053
 -1.3515872  -1.5760187  -0.59295475 -0.66141963  1.3531227   0.47351202
  0.80553395  2.9402657   2.2971811   0.9491949   4.030078    0.6117527
 -0.27292827 -1.1015353  -0.3637588  -1.0808604  -0.38145977 -2.5174286
  1.1072197   2.530051    2.5857248  -1.1959172   1.7793411   0.09467691
 -1.4371932  -2.9791405   1.0630684   0.9067003   0.43923157  3.4134736
  0.47711533 -1.3631905  -3.3734288  -0.2367886   0.3390342  -0.39412883
  0.5333661   0.7227717   0.55354226  0.35878623  0.932402    2.4869206
 -0.55152404  2.4435148  -0.3364468  -1.9685099  -1.7815394  -0.6749915
  2.4418046  -2.3903039   1.1918342   0.36871168 -1.7374214   0.7597355
  1.6034806  -0.9551855   0.8539238  -1.5502286   0.8968359  -0.62780535
 -1.7566789   4.206336    0.92475533  0.4445385   1.4446664   

In [5]:
#
def read_data(file_name):
    with open(file_name, 'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count = {};
        word_index={};

        word2vector = {}
        count=0;
        for line in f:
            line_ = line.strip()  # Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word_index[words_Vec[0]]=count;
            count=count+1;

            word2vector[words_Vec[0]] = np.array(words_Vec[1:], dtype=float)
    print("Total Words in DataSet:", len(word_vocab))
    return word_vocab, word2vector, word_index

In [6]:
def load_embeddings(filename):
    dvocab, w2v,word_index  = read_data(filename);
    return w2v,word_index

In [7]:

embedding_array,word_index = load_embeddings(embedding_filename)
print("Loaded embeddings")

# words_data=pd.read_csv("words_data.csv");
# catchphrases_data=pd.read_csv("catch_data.csv")
print("reading data completed");

Total Words in DataSet: 400000
Loaded embeddings
reading data completed


In [8]:
#sen2vec embeddings
def sen2vec(sentences):
    embed = hub.Module(module_url)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        sentenceEmbeddings = session.run(embed(sentences))    
        return sentenceEmbeddings

# sentence_embeddings = sen2Vec(['UWA was ordered to pay the costs of Dr Gra','Sirtex succeeded in its cross-claim against Dr'])

In [9]:
print(word_index['unk'])


201534


In [10]:
def get_statements(file):
    with open(dirpath + file, 'r',encoding="utf-8", errors='replace') as f:
        data=str(f.read());
        data=data.lower()
        data = data.replace("\"id=", "id=\"");
        data=data.replace("\n","")
        data=data.replace('".*?=.*?"', "",)
        data=data.replace("&","");
        xml = ET.fromstring(str(data))
        name=None;
        rows_list=[];
        catchphrases=[];
        sentences=[];
        for child in xml:
            if child.tag=="catchphrases":
                for catchphrase in child:
                    id=catchphrase.attrib.get("id")
                    #print(catchphrase.text)
                    catchphrases.append({"file_id":file,"Name":name,"Id":id,"text":catchphrase.text})
                    #catchphrases+=tokenizer.tokenize(catchphrase.text)
            if child.tag=="sentences":
                for sentence in child:
                    id = sentence.attrib.get("id")
                    sentences.append({"file_id":file,"Name":name,"Id":id,"text":sentence.text})
                    #sentences+=tokenizer.tokenize(sentence.text)
    
    
    #sentences = [w for w in sentences if not w in stop_words_list] 
    #catchphrases = [w for w in catchphrases if not w in stop_words_list]
    
    
    #sentences=[word.lower() for word in sentences if word.isalphanum()]
    #catchphrases=[word.lower() for word in catchphrases if word.isalphanum()]
    
    return sentences,catchphrases




    

In [11]:
sentences,catchphrases=get_statements(files[0])
start=time.time()
sentences, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
print(sentences,catch_words)
print("time-taken:",time.time()-start)

        Id      file_id  is_catchword                                words
0       s0  08_1056.xml             0             [176, 13, 759, 396, 584]
1       s0  08_1056.xml             0               [13, 759, 396, 584, 0]
2       s0  08_1056.xml             0             [759, 396, 584, 0, 3250]
3       s0  08_1056.xml             0             [396, 584, 0, 3250, 845]
4       s0  08_1056.xml             0              [584, 0, 3250, 845, 21]
5       s0  08_1056.xml             0                [0, 3250, 845, 21, 0]
6       s0  08_1056.xml             0              [3250, 845, 21, 0, 221]
7       s0  08_1056.xml             0                 [845, 21, 0, 221, 3]
8       s0  08_1056.xml             0                 [21, 0, 221, 3, 556]
9       s0  08_1056.xml             0                [0, 221, 3, 556, 603]
10      s0  08_1056.xml             0           [221, 3, 556, 603, 201534]
11      s0  08_1056.xml             0            [3, 556, 603, 201534, 98]
12      s0  08_1056.xml  

In [17]:
def prepare_index_matrices(sentence_indexes):
    length=len(sentence_indexes);
    
    sentence_indexes=[word_index['unk'],word_index['unk']]+sentence_indexes+[word_index['unk'],word_index['unk']];    
    out_indices=[]
    for i in range(2,length):
        out_indices.append(sentence_indexes[i-2:i+3]);
        
    return out_indices;

def prepare_label_matrices(labels):
    length=len(labels);
    labels=[0,0]+labels+[0,0];
    out_labels=[];
    
    for i in range(2,length):
        out_labels.append(labels[i]);
        
    return out_labels;
    

In [18]:
def lookup_indexes_labels(sentences,catchphrases):
    
    sentence_indexes=[];
    labels=[];
    
    for word in sentences:
        if word in word_index:
            sentence_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            sentence_indexes.append(word_index['unk']);
            
    for word in sentences:
        if word in catchphrases:
            labels.append(1.0);
        else:
            #print("unknown word",word)
            labels.append(0.0);
            
    return sentence_indexes,labels;

In [19]:
def get_file_sentences(file_name):
    temp_data=full_data[(full_data.file_id==file_name) & full_data['Id'].str.startswith(('s'))]
    return list(temp_data.text.values)[:-1]

In [24]:
import time

class CatchPhraseExtractor(object):

    def __init__(self, graph):
        self.total_loss=0.0
        self.build_graph(graph)

    def build_graph(self, graph):
        print("generating graph");

        with graph.as_default():
            
            #feed_dict = {self.doc_embedding: doc_embedding, self.phrases_indices: phrases_indices, self.phrase_labels: phrases_label_matrix,
            #self.sentence_indices:sentence_ids_to_indices, self.sentence_embeddings: sentence_embeddings}            
            
            
            self.phrases_indices = tf.placeholder(tf.int32, shape=[None,5])
            self.phrase_labels = tf.placeholder(tf.int32, shape=[None,1])
            
            W = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            phrase_tensor = tf.nn.embedding_lookup(W, self.phrases_indices)
            
            self.sentence_ids_to_indices = tf.placeholder(tf.int32, shape=[None,1])
            self.sentence_embeddings = tf.placeholder(tf.float32, shape=[None,200])
            sentence_tensor = tf.nn.embedding_lookup(self.sentence_embeddings, self.sentence_ids_to_indices)
            sentence_tensor=tf.reshape(sentence_tensor, [tf.shape(sentence_tensor)[0], 200])
            
                                                   
            self.doc_embedding = tf.placeholder(tf.float32, shape=[1,200])
            #document_tensor = tf.convert_to_tensor(np.array([self.doc_embedding for i in range(tf.shape(self.phrases_indices)[0])]))
                     
            phrase_tensor = tf.reshape(phrase_tensor, [tf.shape(phrase_tensor)[0], 1000])
            conv_filter = tf.Variable(tf.truncated_normal((1000, 200), stddev=0.1));
            word_tensor = tf.nn.relu(tf.matmul(phrase_tensor, conv_filter));
            
            #stacked_tensor = tf.concat(values=[word_tensor, sentence_tensor, document_tensor], axis=1)
            stacked_tensor = tf.concat(values=[word_tensor, sentence_tensor], axis=1)
                                                   
            weights_input = tf.Variable(tf.random_normal((400, 100), stddev=0.2));
            weights_input_2 = tf.Variable(tf.random_normal((100, 1), stddev=0.2));
            
            bias1 = tf.Variable(tf.random_normal((1, 100), stddev=0.2));
            bias2 = tf.Variable(tf.random_normal((1, 1), stddev=0.2));
            
            self.sentence_values=self.MLP(stacked_tensor, weights_input, weights_input_2, bias1, bias2);
            
            prediction_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.phrase_labels,logits=self.sentence_values)
            self.loss=tf.reduce_mean(prediction_loss)
                               
            #using the gradient descent optimizer
            optimizer = tf.train.GradientDescentOptimizer(0.1)
            grads = optimizer.compute_gradients(self.loss)
            #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
            self.app = optimizer.apply_gradients(grads)

            self.init = tf.global_variables_initializer()
            
    def MLP(self, stacked_tensor, weights_input, weights_input_2, bias1, bias2):
        h1 = (tf.matmul(weights_input,stacked_tensor,transpose_a=True,transpose_b=True))
        h1 = tf.add(h1, bias1)
        h1 = tf.nn.tanh(h1)
        h2 = tf.matmul(weights_input_2,h1,transpose_a=True,transpose_b=False)
        h2 = tf.add(h2, bias2)
        h2 = tf.nn.sigmoid(h2)
        return tf.transpose(h2)
    
    
    def gradient_descent(self,total_loss):
        total_loss=tf.convert_to_tensor(total_loss)
        with tf.GradientTape(persistent=True) as tape:
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            print(variables)
        return total_loss, tape.gradient(tf.convert_to_tensor(total_loss), variables)
        
                                                   
    def train(self, sess, num_steps):
        self.init.run()
        print("Initailized")
        start=time.time()
        total_loss=0.0;
        for file in files[:1]:
            sentences,catchphrases=get_statements(file);
            file_data_frame, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
            print("Time taken:",time.time()-start)
            
            #needs sentences list
            sentences_list = fp.get_sentences(file) #indexes/order preserved
            sentence_embeddings = sen2vec(sentences_list)
            
            print("Time taken:",time.time()-start)
            sentence_ids_to_indices = np.array([int(x[1:]) for x in list(file_data_frame['Id'].values)])
            
            
            #phrase inputs and labels
            phrases_indices = np.array(list(file_dataframe['words'].values))
            phrases_label_matrix = np.array(list(file_dataframe['is_catchword'].values))
            
            
            #doc embedding
            doc_embedding = d2v_model.docvecs[file]
            
            
            feed_dict = {self.doc_embedding: doc_embedding, self.phrases_indices: phrases_indices, self.phrase_labels: phrases_label_matrix, self.sentence_indices:sentence_ids_to_indices, self.sentence_embeddings: sentence_embeddings}            
            print("sending inputs");
            #loss_val = sess.run([self.loss], feed_dict=feed_dict)  
            _,loss_val,outputs = sess.run([self.app,self.loss,self.sentence_values], feed_dict=feed_dict)

            print("Train Finished: loss_val=", loss_val)
            for i in range(len(outputs)):
                print(outputs[i])
            
        
    def load_embeddings(self,embedding_array):
        sess.run(self.embedding_init, feed_dict={self.embedding_placeholder: np.asarray(list(embedding_array.values()))})
        

    

In [25]:
def get_features(inputs):
    out=np.zeros((len(inputs),200));
    for i in range(len(inputs)):
        try:
            c = w2v[inputs[i]]
        except KeyError:
            c = np.zeros((1,200));
        out[i]=c;
    
    return np.transpose(out);

In [26]:
graph = tf.Graph()

model = CatchPhraseExtractor(graph)
start=time.time()

#print(fp.get_dataframe(files[0]))
#sentences_list = fp.get_file_sentences(files[0]) #indexes/order preserved
#sentence_embeddings = sen2vec(sentences_list)
end=time.time()
print("time taken:",end-start);

generating graph
time taken: 2.4080276489257812e-05


In [27]:
with tf.Session(graph=graph) as sess:
  
    #model.load_embeddings(embedding_array)    
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    #global_step = tf.train.get_or_create_global_step()
    for i in range(1):
        model.train(sess, 10)
    #optimizer.apply_gradients(zip(grads, model.variables), global_step)
        

Initailized
Time taken: 4.4289469718933105


NotFoundError: /var/folders/38/3kwpqlj11153rrh6wl934p5r0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/tfhub_module.pb; No such file or directory